# CDAT Migration Regression Testing Notebook (`.nc` files)

This notebook is used to perform regression testing between the development and
production versions of a diagnostic set.

## How it works

It compares the relative differences (%) between ref and test variables between
the dev and `main` branches.

## How to use

PREREQUISITE: The diagnostic set's netCDF stored in `.json` files in two directories
(dev and `main` branches).

1. Make a copy of this notebook under `auxiliary_tools/cdat_regression_testing/<DIR_NAME>`.
2. Run `mamba create -n cdat_regression_test -y -c conda-forge "python<3.12" xarray netcdf4 dask pandas matplotlib-base ipykernel`
3. Run `mamba activate cdat_regression_test`
4. Update `SET_DIR` and `SET_NAME` in the copy of your notebook.
5. Run all cells IN ORDER.
6. Review results for any outstanding differences (>=1e-5 relative tolerance).
   - Debug these differences (e.g., bug in metrics functions, incorrect variable references, etc.)


## Setup Code


In [ ]:
import glob
from typing import List

import numpy as np
import xarray as xr

from e3sm_diags.derivations.derivations import DERIVED_VARIABLES

DEV_DIR = "24-12-09-main"
DEV_DIR = f"/global/cfs/cdirs/e3sm/www/e3sm_diags/complete_run/{DEV_DIR}/"
DEV_GLOB = sorted(glob.glob(DEV_DIR + "**/**/*.nc"))
DEV_NUM_FILES = len(DEV_GLOB)

MAIN_DIR = "v2.12.1"
MAIN_PATH = f"/global/cfs/cdirs/e3sm/www/e3sm_diags/complete_run/{MAIN_DIR}/"
MAIN_GLOB = sorted(glob.glob(MAIN_DIR + "**/**/*.nc"))
MAIN_NUM_FILES = len(MAIN_GLOB)


# def _remove_unwanted_files(file_glob: List[str]) -> List[str]:
#     """Remove files that we don't want to compare.

#     * area_mean_time_series -- `main` does not generate netCDF
#     * enso_diags -- `main` does not generate netCDF
#     * qbo -- variable name differs
#     * diurnal_cycle -- variable name differs
#     * diff -- comparing the difference between regridded files is not helpful
#       between branches because of the influence in floating point errors.
#     * ERA5_ext-U10-ANN-global_ref and ERA5_ext-U10-JJA-global_ref -- dev
#       branch does not generate these files because it is a model-only run.

#     Parameters
#     ----------
#     file_glob : List[str]
#         _description_

#     Returns
#     -------
#     List[str]
#         _description_
#     """

#     new_glob = []

#     for fp in file_glob:
#         if (
#             "area_mean_time_series" in fp
#             or "enso_diags" in fp
#             or "qbo" in fp
#             or "diurnal_cycle" in fp
#             or "diff" in fp
#             or "ERA5_ext-U10-ANN-global_ref" in fp
#             or "ERA5_ext-U10-JJA-global_ref" in fp
#         ):
#             continue

#         new_glob.append(fp)

#     return new_glob


# DEV_GLOB = _remove_unwanted_files(DEV_GLOB)
# MAIN_GLOB = _remove_unwanted_files(MAIN_GLOB)

In [8]:
MAIN_NUM_FILES

0

In [2]:
MAIN_GLOB

[]

In [3]:
MAIN_NUM_FILES

0

In [ ]:
def _get_relative_diffs():
    # We are mainly focusing on relative tolerance here (in percentage terms).
    atol = 0
    rtol = 1e-4

    results = {
        "missing_files": [],
        "missing_vars": [],
        "matching_files": [],
        "mismatch_errors": [],
        "not_equal_errors": [],
        "key_errors": [],
    }

    for fp_main in MAIN_GLOB:
        fp_dev = fp_main.replace(MAIN_DIR, DEV_DIR)

        print("Comparing:")
        print(f"    * {fp_dev}")
        print(f"    * {fp_main}")

        if "annual_cycle_zonal_mean" in fp_main:
            if "test.nc" in fp_main:
                fp_dev = fp_dev.replace("test.nc", "ref.nc")
            elif "ref.nc" in fp_main:
                fp_dev = fp_dev.replace("ref.nc", "test.nc")

        try:
            ds1 = xr.open_dataset(fp_dev)
            ds2 = xr.open_dataset(fp_main)
        except FileNotFoundError as e:
            print(f"    {e}")

            if isinstance(e, FileNotFoundError) or isinstance(e, OSError):
                results["missing_files"].append(fp_dev)

            continue

        var_key = fp_main.split("-")[-3]

        # for 3d vars such as T-200
        var_key.isdigit()
        if var_key.isdigit():
            var_key = fp_main.split("-")[-4]

        dev_data = _get_var_data(ds1, var_key)
        main_data = _get_var_data(ds2, var_key)

        print(f"    * var_key: {var_key}")

        if dev_data is None or main_data is None:
            if dev_data is None:
                results["missing_vars"].append(fp_dev)
            elif main_data is None:
                results["missing_vars"].append(fp_main)

            print("    * Could not find variable key in the dataset(s)")

            continue

        try:
            np.testing.assert_allclose(
                dev_data,
                main_data,
                atol=atol,
                rtol=rtol,
            )
            results["matching_files"].append(fp_main)
        except (KeyError, AssertionError) as e:
            msg = str(e)

            print(f"    {msg}")

            if "mismatch" in msg:
                results["mismatch_errors"].append(fp_dev)
            elif "Not equal to tolerance" in msg:
                results["not_equal_errors"].append(fp_dev)
        else:
            print(f"    * All close and within relative tolerance ({rtol})")

    return results


def _get_var_data(ds: xr.Dataset, var_key: str) -> np.ndarray:
    """Get the variable data using a list of matching keys.

    The `main` branch saves the dataset using the original variable name,
    while the dev branch saves the variable with the derived variable name.
    The dev branch is performing the expected behavior here.

    Parameters
    ----------
    ds : xr.Dataset
        _description_
    var_key : str
        _description_

    Returns
    -------
    np.ndarray
        _description_
    """

    data = None

    try:
        data = ds[var_key].values
    except KeyError:
        try:
            var_keys = DERIVED_VARIABLES[var_key].keys()
        except KeyError:
            var_keys = DERIVED_VARIABLES[var_key.upper()].keys()

        var_keys = [var_key] + list(sum(var_keys, ()))

        for key in var_keys:
            if key in ds.data_vars.keys():
                data = ds[key].values
                break

    return data

In [ ]:
def _check_if_files_found():
    if DEV_NUM_FILES == 0 or MAIN_NUM_FILES == 0:
        raise IOError(
            "No files found at DEV_DIR and/or MAIN_DIR. "
            f"Please check {DEV_DIR} and {MAIN_DIR}."
        )


def _check_if_matching_filecount():
    if DEV_NUM_FILES != MAIN_NUM_FILES:
        raise IOError(
            "Number of files do not match at DEV_DIR and MAIN_DIR "
            f"({DEV_NUM_FILES} vs. {MAIN_NUM_FILES})."
        )

    print(f"Matching file count ({DEV_NUM_FILES} and {MAIN_NUM_FILES}).")


def _check_if_missing_files():
    missing_dev_files = []
    missing_main_files = []

    for fp_main in MAIN_GLOB:
        fp_dev = fp_main.replace(MAIN_DIR, DEV_DIR)

        if fp_dev not in DEV_GLOB:
            missing_dev_files.append(fp_dev)

    for fp_dev in DEV_GLOB:
        fp_main = fp_dev.replace(DEV_DIR, MAIN_DIR)

        if fp_main not in MAIN_GLOB:
            missing_main_files.append(fp_main)

    return missing_dev_files, missing_main_files

## 1. Check for matching and equal number of files


In [ ]:
_check_if_files_found()

In [ ]:
DEV_GLOB = [fp for fp in DEV_GLOB if "diff.nc" not in fp]
MAIN_GLOB = [fp for fp in MAIN_GLOB if "diff.nc" not in fp]

In [ ]:
len(DEV_GLOB), len(MAIN_GLOB)

In [ ]:
missing_dev_files, missing_main_files = _check_if_missing_files()

print(f"Missing dev files: {len(missing_dev_files)}")
print(f"Missing main files: {len(missing_main_files)}")

### Check missing main files (not concerned)

Results:

- The missing files are due to a recent .cfg update in [PR #830](https://github.com/E3SM-Project/e3sm_diags/pull/830)


In [ ]:
missing_main_files

### Check missing dev files:

Results:

- The missing reference files are due to not saving them out to netCDF since they are the same as the test files (skipped, model-only run)


In [ ]:
missing_dev_files

## 2 Compare the netCDF files between branches

- Compare "ref" and "test" files
- "diff" files are ignored because getting relative diffs for these does not make sense (relative diff will be above tolerance)


In [ ]:
results = _get_relative_diffs()

In [ ]:
import pandas as pd

# Statistics
(
    missing_files,
    missing_vars,
    matching_files,
    mismatch_errors,
    not_equal_errors,
    key_errors,
) = results.values()

### Missing Files


In [ ]:
missing_files

- AOD_550 files were retired (https://github.com/E3SM-Project/e3sm_diags/issues/852)


In [ ]:
missing_files = []

### `NaN` Mismatching Errors

I found these `nan` mismatch errors occur due to either:

1. Regional subsetting on "ccb" flag in CDAT adding a coordinate points -- removing these coordinates results in matching results
2. Slightly different masking in the data between xCDAT and CDAT via xESMF/ESMF -- same number of nans just slightly shifted over some coordinates points

- Refer to PR [#794](https://github.com/E3SM-Project/e3sm_diags/pull/794)


In [ ]:
mismatch_errors = [
    f
    for f in mismatch_errors
    # https://github.com/E3SM-Project/e3sm_diags/pull/794
    if "TAUXY" not in f and "ERA5-TREFHT" not in f and "MERRA2-TREFHT" not in f
    # https://github.com/E3SM-Project/e3sm_diags/pull/798#issuecomment-2251287986
    and "ceres_ebaf_toa_v4.1-ALBEDO" not in f
]

In [ ]:
mismatch_errors

### Not Equal Errors

- Note, some files are omitted due to known root causes to the diffs (not a concern)


In [ ]:
not_equal_errors = [
    f
    for f in not_equal_errors
    # https://github.com/E3SM-Project/e3sm_diags/issues/797
    if "MISRCOSP-CLDLOW_TAU1.3_9.4_MISR" not in f
    and "MISRCOSP-CLDLOW_TAU1.3_MISR" not in f
    and "MISRCOSP-CLDLOW_TAU9.4_MISR" not in f
    # only 1 mismatching element with 1e-4 tolerance
    and "ERA5-OMEGA-JJA" not in f and "MERRA2-OMEGA-JJA" not in f
    # https://github.com/E3SM-Project/e3sm_diags/issues/787
    and "MERRA2-U" not in f
    # https://github.com/E3SM-Project/e3sm_diags/issues/852
    and "AOD_550" not in f
]

In [ ]:
not_equal_errors

Note, the two files have less than 10 elements that mismatch.
This does not seem to be of a concern since the rtol is 0.04% and 1.2%.


```python
Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/843-migration-phase3-model-vs-obs/lat_lon/Cloud MISR/MISRCOSP-CLDTOT_TAU1.3_9.4_MISR-ANN-global_test.nc
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/lat_lon/Cloud MISR/MISRCOSP-CLDTOT_TAU1.3_9.4_MISR-ANN-global_test.nc
    * var_key: CLDTOT_TAU1.3_9.4_MISR

Not equal to tolerance rtol=0.0001, atol=0

Mismatched elements: 7 / 64800 (0.0108%)
Max absolute difference: 0.0970192
Max relative difference: 0.00456364
 x: array([[ 4.128816,  4.128816,  4.128816, ...,  7.77411 ,  7.77411 ,
         7.77411 ],
       [ 4.164101,  4.164073,  4.164018, ...,  7.538528,  7.538158,...
 y: array([[ 4.128816,  4.128816,  4.128816, ...,  7.774109,  7.774109,
         7.774109],
       [ 4.164101,  4.164073,  4.164018, ...,  7.538528,  7.538157,...

Not equal to tolerance rtol=0.0001, atol=0

Mismatched elements: 8 / 64800 (0.0123%)
Max absolute difference: 0.0970192
Max relative difference: 0.01244658
 x: array([[ 0.056777,  0.056777,  0.056777, ...,  1.274017,  1.274017,
         1.274017],
       [ 0.207892,  0.207774,  0.207536, ...,  1.675944,  1.676576,...
 y: array([[ 0.056777,  0.056777,  0.056777, ...,  1.274017,  1.274017,
         1.274017],
       [ 0.207892,  0.207774,  0.207536, ...,  1.675944,  1.676576,...
```


In [ ]:
ds1 = xr.open_dataset(
    "/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/843-migration-phase3-model-vs-obs/lat_lon/Cloud MISR/MISRCOSP-CLDTOT_TAU1.3_9.4_MISR-ANN-global_test.nc"
)
ds2 = xr.open_dataset(
    "/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/lat_lon/Cloud MISR/MISRCOSP-CLDTOT_TAU1.3_9.4_MISR-ANN-global_test.nc"
)

In [ ]:
np.testing.assert_allclose(
    ds1["CLDTOT_TAU1.3_9.4_MISR"], ds2["CLDTOT_TAU1.3_9.4_MISR"], atol=0, rtol=1e-4
)

In [ ]:
ds1["CLDTOT_TAU1.3_9.4_MISR"].values.sum()

In [ ]:
ds2["CLDTOT_TAU1.3_9.4_MISR"].values.sum()

In [ ]:
# Assuming these variables are defined in your notebook
matching_files_count = len(matching_files)
missing_vars_count = len(missing_vars)
mismatch_errors_count = len(mismatch_errors)
not_equal_errors_count = len(not_equal_errors)
key_errors_count = len(key_errors)
missing_files_count = len(missing_files)

sum_files_compared = (
    matching_files_count
    + missing_vars_count
    + mismatch_errors_count
    + not_equal_errors_count
    + key_errors_count
    + missing_files_count
)

pct_match = (matching_files_count / sum_files_compared) * 100

# Collect statistics into a dictionary
statistics = {
    "stat_name": [
        "matching_files_count",
        "missing_vars_count",
        "mismatch_errors_count",
        "not_equal_errors_count",
        "key_errors_count",
        "missing_files_count",
    ],
    "value": [
        matching_files_count,
        missing_vars_count,
        mismatch_errors_count,
        not_equal_errors_count,
        key_errors_count,
        missing_files_count,
    ],
    "pct": [
        matching_files_count / sum_files_compared,
        missing_vars_count / sum_files_compared,
        mismatch_errors_count / sum_files_compared,
        not_equal_errors_count / sum_files_compared,
        key_errors_count / sum_files_compared,
        missing_files_count / sum_files_compared,
    ],
}

# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame(statistics)

# Display the DataFrame
print(df)